In [24]:
import csv
import time
import requests
import pandas as pd
import plotly.express as px
from pandas import json_normalize

In [23]:
def API_Call(bdate, edate, state, param):
    print("Running...")
    
    try:
        email = "shervorn.mathews@yahoo.com"
        key = "russetbird18"
        url = f'https://aqs.epa.gov/data/api/dailyData/byState?email={email}&key={key}&param={param}&bdate={bdate}&edate={edate}&state={state}'

        response = requests.get(url)
        
        JSON_To_DF(response)
    except:
        print("API call may have failed.\nRetrying in 10 minutes.")
        print(url)
        time.sleep(60)
        print("Retrying...")
        API_Call(bdate, edate, state)

In [5]:
def JSON_To_DF(response):
    data = response.json()

    recs = data['Data']

    starter_df = json_normalize(recs)
    
    DF_Cleaner(starter_df)

| State | Abbr | FIPS Code |
| --- | ----------- | --- |
| California | CA | 06 |
| Paragraph | Text | Blah |

In [19]:
def DF_Cleaner(starter_df):
    global MainDF
    
    df = starter_df

    colToDrop = ['site_number', 'poc', 'datum', 'sample_duration_code', 'sample_duration',
                'state_code', 'county_code', 'parameter_code', 'method_code', 'method',
                'site_address', 'city', 'cbsa_code', 'cbsa', 'date_of_last_change', 'local_site_name',
                'pollutant_standard', 'event_type', 'first_max_hour', 'validity_indicator',
                'observation_percent', 'observation_count', 'arithmetic_mean']

    df = df.drop(columns= colToDrop, axis = 1).convert_dtypes().drop_duplicates()

    df = df.astype({"aqi": str})

    df["date_local"] = pd.to_datetime(df["date_local"])

    df = df[df.aqi != "<NA>"]

    df = df.astype({"aqi": int})
    
    MainDF = MainDF.append(df)
    
    print("Done!")

In [22]:
MainDF = pd.DataFrame()

In [30]:
bdate = 20220602
edate = 20221231
state = "06"
# param = "88502"
# param = "88101"
param = "88502,42102,42101"

#Do 2020 to 2022

# 20200101
# 20210602

In [31]:
API_Call(bdate, edate, state, param)
#Pause for 5 mins between API calls
# time.sleep(300)

Running...
Done!


C:\Users\nrovr\AppData\Local\Temp\ipykernel_26952\2892282483.py:22: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.



In [33]:
MainDF.shape

# MainDF.to_csv('cleaned_AQI.csv')

In [13]:
print(MainDF['parameter'].value_counts())

Carbon monoxide                           9704
Acceptable PM2.5 AQI & Speciation Mass    9505
Name: parameter, dtype: Int64


In [12]:
fig = px.line(MainDF, x='date_local', y='aqi',
             color='county', 
            #  labels={
            #       "data_value": "Percent (%)",
            #       "yearstart": "Year"
            #   }, 
             title='Test Plot')
fig.show()